Validação cruzada
=================



## O problema do vazamento de dados



Vamos pensar em uma situação hipotética: você tem um certo conjunto de dados (features + target) e quer treinar uma árvore de decisão usando estes dados. Você se recorda que é necessário dividir seu conjunto de dados em treino e teste e faz exatamente isso antes de seguir em frente (usando uma semente aleatória para que sua divisão seja reprodutível). Na hora de treinar seu modelo, você decide utilizar os hiperparâmetros padrão do `scikit-learn`, porém usa uma semente aleatória pois tem interesse em reproduzir seus resultados. Você segue então com o ajuste do seu modelo usando os dados de treino e em seguida decide ver como seu modelo se comporta com dados novos ao tentar prever os dados de teste que não foram &ldquo;vistos&rdquo; durante o treino.

Esse é um cenário que já foi explorado neste curso, nada de novo até agora.

Digamos que você está curiosa e retorna e altere a semente aleatória para um outro valor, repetindo todo o processo de split, treino e teste com a nova semente. Como você alterou a semente aleatória, outros dados serão selecionados no split de treino e teste e outra árvore de decisão será induzida durante o treino do modelo. Logo, a performance do modelo no conjunto de teste muito provavelmente será diferente, podendo ser melhor ou pior.

Até aqui nada surpreendente.

Agora imagina que você cria um laço de repetição que testa 1000000 valores de semente aleatória diferentes para realizar o mesmo processo de split, treino e teste descrito acima. Para cada semente testada, você armazena o valor da semente e a performance do modelo (o valor de RMSE, por exemplo). O que você espera que aconteça?

O resultado é que, por puro acaso, alguns modelos irão apresentar métricas melhores que outros. A pergunta é: <u>seria uma boa ideia selecionar o modelo que teve a melhor métrica, pois esse é o modelo que melhor generaliza seu problema</u>?

A resposta é **NÃO**. Ao fazer esse exercício de selecionar &ldquo;a melhor semente aleatória&rdquo;, você está permitindo que o acaso faça sua magia e te engane que você encontrou um modelo melhor que os outros, quando na verdade você fez um processo chamado de <u>vazamento de dados</u> (*data leakage* em inglês) que aumentou o sobreajuste do seu modelo e te enganou sobre isso (tudo ao mesmo tempo!).

*Data leakage* é quando a informação do conjunto de teste &ldquo;vaza&rdquo; no seu treino e leva você a sobreajustar seu modelo sem saber que está sobreajustando! Ao selecionar &ldquo;a melhor semente aleatória&rdquo;, você está usando a informação da métrica no conjunto de teste para realizar a escolha do seu modelo, o que faz com que a informação do conjunto de teste seja usada na seleção do modelo o que é um **grande NÃO-NÃO**.

Vamos repetir pois isso é muito importante: <u>você não pode usar o conjunto de teste no processo de escolha do modelo</u>. Aliás, você não pode usar o conjunto de teste para nada, a não ser o teste final do eventual modelo selecionado. (Isso quer dizer que também não podemos usar os dados de teste para normalizações!)

Até agora, nos notebooks anteriores, nós usamos as métricas do conjunto de teste para *observar* como diferentes modelos se comportam, mas essa **não** pode ser nossa estratégia para selecionar &ldquo;o melhor modelo&rdquo; para nossos dados.

Mas&#x2026; se não podemos usar o conjunto de teste e podemos ter o azar de sermos contemplados com uma semente aleatória que nos leva ao sobreajuste, o que podemos fazer para nos sentirmos mais confiantes nos nossos modelos de machine learning?

A resposta, meus caros, é o método da **Validação Cruzada**.



## Validação Cruzada



Antes de ver o que é a validação cruzada (*cross-validation* em inglês), vamos ver onde que ela entra dentro do fluxograma de aprendizado de máquina.

![img](https://scikit-learn.org/stable/_images/grid_search_workflow.png)

Veja que a validação cruzada é uma estratégia que usamos para encontrar um conjunto razoavelmente bom de hiperparâmetros do modelo (na imagem está escrito *best parameters*, mas eu diria que o melhor seria se estivesse escrito *reasonably good hyperparameters*).

Existem diversos tipos de validação cruzada, mas todos seguem a mesma ideia geral: os dados de treino não são *todos* utilizados para treinar o modelo, uma fração deles é reservada para o processo de validação do modelo; esta estratégia é repetida algumas vezes para termos uma boa estatística do processo de validação.

Vamos ver um tipo de validação cruzada muito utilizado: a validação $k$-fold. Este método consiste em dividir o conjunto de treino em $k$ conjuntos de dados de tamanho igual (ou o mais próximo possível disso). O modelo sendo validado será treinado $k$ vezes neste processo, cada treino será realizado com $k-1$ destes conjuntos de dados e a performance do modelo será mensurada no conjunto que ficou de fora (chamado aqui de conjunto de validação). Este processo é repetido de forma que todos os $k$ conjuntos sejam o conjunto de validação uma (e apenas uma) vez.

Abaixo temos uma representação visual da validação $k$-fold para o valor de $k=5$.

![img](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

Após a validação $k$-fold teremos uma lista de tamanho $k$ representando a performance dos $k$ modelos que treinamos. A comparação entre diferentes modelos se dá pela média da performance.

Esta estratégia de validação cruzada nos permite comparar algoritmos diferentes (árvores de decisão e regressão linear, por exemplo) bem como algoritmos iguais com diferentes conjuntos de hiperparâmetros (floresta aleatória com 100 ou 1000 árvores de decisão, por exemplo).

Após a comparação dos modelos, o que apresentar as melhores métricas deverá ser escolhido. O próximo passo é então treinar **o modelo semifinal** usando o algoritmo e conjunto de hiperparâmetros escolhidos e *todo* o conjunto de dados de treino. Este **modelo semifinal** é o único que terá a honra de prever os dados de teste. Os demais não poderão ter essa honra para não enviesar a nossa análise e cair novamente no pecado capital do vazamento de dados.

Após a previsão dos dados de teste você deve se perguntar se seu modelo teve a capacidade de generalizar o problema. Isto é, a performance nos dados de teste está dentro do que você consideraria aceitável?

-   Se sim, então você pode finalmente treinar **o modelo final** usando *todos* seus dados (treino + teste); este é o modelo que você usará caso queira fazer algum tipo de previsão futura.

-   Se não, então fim de jogo; você não conseguiu encontrar um bom modelo usando a metodologia proposta&#x2026; se ainda sim quiser usar aprendizado de máquina neste problema deverá retornar desde o início e propor alterações na sua metodologia e realizar o processo todo do zero. Quem sabe suas features não são suficientes para descrever o problema. Quem sabe seus algoritmos não são suficientes para capturar a complexidade dos dados. Quem sabe os hiperparâmetros escolhidos promoveram subajuste ou sobreajuste. Etc&#x2026;

Uma das melhores fontes para se ler sobre validação cruzada é a própria [documentação](https://scikit-learn.org/stable/modules/cross_validation.html) do `scikit-learn`. O processo e diferentes estratégias que temos são muito bem explicadas lá. Infelizmente, o vídeo do StatQuest sobre esse assunto é simples demais e não discute a questão do conjunto de treino e teste. Confiram também [este vídeo](https://youtu.be/-8s9KuNo5SA) que explica muito bem a validação cruzada, mostrando os erros comuns no aprendizado de máquina.



## Usando validação cruzada para comparar diferentes modelos



Vamos usar validação cruzada para comparar diferentes modelos para nosso dataset de diamantes. Neste caso, usaremos a validação $k$-fold considerando um valor de $k=10$. Em geral, o valor de $k$ escolhido costuma ser 5 ou 10.

Primeiramente precisamos carregar os dados.



In [1]:
import seaborn as sns
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()
X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

Para usar a validação $k$-fold usamos a função `cross_val_score`. Para isso, precisamos criar o modelo antes como mostra o código abaixo. Essa função já cuida de tudo para nós! Ela separa os dados em folds, treina o modelo e computa a métrica desejada.



In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

NUM_ARVORES = 10
NUM_FOLDS = 10
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [0.86931393 0.86747536 0.87234042 0.87176931 0.87338384 0.87551236
 0.86697302 0.86332081 0.86679812 0.86782233]

A média dos scores é de:  0.8694709483426829


Quando nenhuma métrica é indicada na função `cross_val_score`, o `scikit-learn` usará a métrica padrão do seu modelo utilizado (costuma ser o $R^2$ para problemas de regressão ou a acurácia para problemas de classificação). Neste caso, os valores acima indicam o $R^2$ do modelo obtido. Como já discutido em outro notebook, a forma como o `scikit-learn` computa o $R^2$ não é a ideal.

Existem diversos scores possíveis que podemos utilizar para a validação cruzada. Você pode checar as que já estão embutidas no `scikit-learn` na [documentação](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values). Digamos que você queira usar o RMSE como score, basta passar a string `"neg_root_mean_squared_error"` no argumento `scoring`, assim como feito abaixo.



In [3]:
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-1402.1516967  -1446.83751054 -1427.34209668 -1422.34538022
 -1439.66733232 -1391.7646204  -1448.98320846 -1443.18534277
 -1496.46828125 -1490.92054804]

A média dos scores é de:  -1440.9666017372572


Ué, como assim o RMSE deu negativo? O $R^2$ tinha funcionado bem (era positivo, como esperado), mas o RMSE estranhamente deu negativo. Isso acontece porque o `scikit-learn` definiu que métricas com o nome de *score* devem todas seguir a mesma regra: quanto maior, melhor! Sabemos que isso é válido para o $R^2$, por isso não tivemos surpresas. Porém, o RMSE é o contrário: quanto menor o RMSE melhor a performance do meu modelo. Para satisfazer a definição de score do `scikit-learn`, devemos usar o negativo do RMSE como score (por isso tem um `neg` na string que passamos no argumento `scoring`, vem de &ldquo;negativo&rdquo;). Basta você remover o sinal de negativo e terá seu RMSE tradicional como de costume. Sabendo disso, podemos dizer que o RMSE da validação cruzada do nosso modelo foi de 1441 dólares.

Agora, vamos ver o que acontece com o RMSE da validação cruzada quando aumentamos o número de árvores de decisão de 10 para 100.



In [4]:
NUM_ARVORES = 100
NUM_FOLDS = 10
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-1341.55701171 -1394.39388685 -1352.1163828  -1369.27057055
 -1378.84199415 -1339.60033271 -1387.3036807  -1409.48832346
 -1445.31889204 -1427.7959096 ]

A média dos scores é de:  -1384.5686984565775


O RMSE da validação cruzada deste modelo é 1385 dólares. Considerando apenas estes dois resultados, qual dos dois modelos você escolheria? Se respondeu o modelo com 100 árvores de decisão então você acertou! 🎉

Observe que não utilizamos o conjunto de teste até agora! Os scores foram computados apenas usando os dados de treino.

Podemos repetir este procedimento quantas vezes quisermos para testar outras combinações de modelos e hiperparâmetros. A maior limitação dessa estratégia é que ela é computacionalmente custosa&#x2026; uma estratégia possível para contornar essa desvantagem é testar um grande número de hiperparâmetros *sem* usar validação cruzada (apenas no conjunto de treino, como sempre!) e depois realizar a validação cruzada apenas para aqueles conjuntos de hiperparâmetros que tiverem a melhor performance.

Vamos supor que fizemos isso e chegamos na conclusão que o melhor modelo para nós é de fato a floresta aleatória com 100 árvores de decisão e o restante dos hiperparâmetros com o valor padrão do `scikit-learn`. Agora, e somente agora, é que nós podemos usar nosso conjunto de teste para inferir a performance do modelo.



In [5]:
from sklearn.metrics import mean_squared_error

NUM_ARVORES = 100
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
modelo_rf.fit(X_treino, y_treino)

y_previsao = modelo_rf.predict(X_teste)
RMSE = mean_squared_error(y_teste, y_previsao, squared=False)

print(f"O RMSE do modelo floresta aleatória no conjunto de teste foi de {RMSE} dólares.")

O RMSE do modelo floresta aleatória no conjunto de teste foi de 1384.6366214316938 dólares.


Observamos que o RMSE para o conjunto de teste foi de 1385 dólares, similar ao que obtivemos na validação cruzada. Esse resultado é bom no sentido que nossa estratégia de validação cruzada conseguiu capturar bem a performance do modelo em dados que ele nunca viu (lembre-se que o conjunto de teste não foi utilizado para nada até agora!). <u>Isso suporta a hipótese de que não houve overfit</u>. Observaríamos overfit caso a performance no conjunto de teste fosse significantemente pior que a performance na validação cruzada. Não é nada usual a métrica no conjunto de teste ser significantemente melhor do que a obtida no conjunto de treino.

Agora, esse resultado nos diz que nosso modelo é um **bom modelo**? Não sei, isso vai depender se essa performance que obtivemos é considerada boa dentro do universo onde os dados estão inseridos. Como muito provavelmente ninguém aqui é um especialista em diamantes, é difícil dizer se um RMSE de 1385 dólares é objetivamente bom ou ruim. Precisamos nos debruçar mais sobre o problema para responder essa pergunta com propriedade, mas uma opinião de leigo me sugere que não é um valor bom.



## Pipelines



Digamos que você queira fazer um estudo similar ao que fizemos acima, porém utilizando o modelo $k$-NN. Sabemos que o modelo $k$-NN é baseado em distâncias e que modelos baseados em distâncias são prejudicados quando os dados não estão normalizados.

Sabemos também que não podemos fazer nada com o conjunto de dados de teste até o final de *toda* a nossa análise. Isso inclui o processo de normalização! Só podemos normalizar os dados considerando a informação dos dados de treino.

Nosso processo aqui então é um processo composto: normalizar e treinar um modelo.

O `scikit-learn` tem uma forma de combinar processos chamada de `Pipeline` que atua como uma tubulação de informação: a informação que entra no `Pipeline` é transmitida para todas as componentes do `Pipeline` até o fim. Uma forma de criar um `Pipeline` é usando o `make_pipeline` e passando para ele todas as componentes do modelo.

Vamos ver um exemplo.



In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

NUM_VIZINHOS = 3
NUM_FOLDS = 10

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)

scores = cross_val_score(
    modelo_knn_composto,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-1492.58797404 -1536.344317   -1490.72538853 -1519.26555191
 -1549.38567295 -1514.19954687 -1501.67522334 -1538.64171942
 -1599.10513296 -1549.31875221]

A média dos scores é de:  -1529.1249279225076


Nosso `modelo_knn_composto` já cuida para nós da normalização (usando `MinMaxScaler`) e do ajuste do modelo $k$-NN. Vamos comparar esse modelo com $k=3$ vizinhos com um com $k=7$ vizinhos.



In [7]:
NUM_VIZINHOS = 7
NUM_FOLDS = 10

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)

scores = cross_val_score(
    modelo_knn_composto,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-1403.43840774 -1432.05381302 -1389.05961224 -1415.83127843
 -1443.82191369 -1408.20343265 -1424.81006271 -1460.45595828
 -1515.36550445 -1444.70361755]

A média dos scores é de:  -1433.7743600754025


Com $k=3$ vizinhos obtivemos um RMSE de 1529 dólares na validação cruzada. Já com $k=7$ obtivemos um RMSE menor de 1434 dólares. Logo, após esse experimento simples nós concluimos que o modelo $k$-NN com $k=7$ vizinhos teve uma performance melhor e será escolhido para ter a honra de prever o conjunto de teste. <u>Nota</u>: apenas dois experimentos é muito pouco, devemos fazer mais! Mas aqui estamos apenas mostrando o raciocínio.

Para treinar um modelo composto nós usamos o `fit` da mesma maneira que j́a fizemos.



In [8]:
NUM_VIZINHOS = 7

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)
modelo_knn_composto.fit(X_treino, y_treino)

y_previsao = modelo_knn_composto.predict(X_teste)
RMSE = mean_squared_error(y_teste, y_previsao, squared=False)

print(f"O RMSE do modelo k-NN no conjunto de teste foi de {RMSE} dólares.")

O RMSE do modelo k-NN no conjunto de teste foi de 1436.3964946278834 dólares.


Novamente o RMSE do modelo no conjunto de teste foi muito similar ao RMSE obtido na validação cruzada. A mesma discussão realizada acima cabe aqui.

Para saber mais sobre `Pipelines`, veja a [documentação](https://scikit-learn.org/stable/modules/compose.html).



## Outros tipos de validação cruzada



A validação $k$-fold é a mais usual, porém existem outras estratégias de validação cruzada. Algumas que valem a pena ter ciência:

-   **Leave one out** (LOO): estratégia similar à validação $k$-fold, porém com o valor de $k$ sendo igual ao número de exemplos disponíveis. É uma estratégia interessante quando se tem poucos dados. [Documentação](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html).

-   **Stratified $k$-fold**: estratégia similar à validação $k$-fold para ser usada em problemas de classificação. A ideia é manter a proporção dos rótulos quando se efetuar a divisão dos folds. É particularmente boa em conjuntos de dados desbalanceados (isto é, com quantidades significativamente diferentes dos rótulos). [Documentação](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html).

-   **Time series split**: estratégia para se dividir séries temporais. A ideia é que cada segmentação dos dados nós teremos um conjunto de teste que formam um bloco no eixo do tempo. Todos os dados com tempo menor do que os do bloco de teste são considerados o conjunto de treino. Todos os dados com tempo maior que o conjunto de teste não são considerados neste split. Veja a imagem abaixo. Se seus dados variam ao longo do tempo, então é melhor utilizar esta estratégia do que a validação $k$-fold. [Documentação](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html).

![img](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png)



## XKCD relevante



![img](https://imgs.xkcd.com/comics/significant.png)

`Imagem: Significant (XKCD) disponível em https://xkcd.com/882`



## Referências e leitura adicional



1.  [https://scikit-learn.org/stable/modules/cross_validation.html](https://scikit-learn.org/stable/modules/cross_validation.html)
2.  [https://scikit-learn.org/stable/modules/compose.html](https://scikit-learn.org/stable/modules/compose.html)

